We show how to compute the symmetrized product of two symmetric matrices $A$, $B$, defined by $C=AB+BA$ using ${n+2 \choose 3}$ multiplications rather than $n^3$. Some third order intermediate tensors are computed using the Cyclops Tensor Framework (CTF) library

In [1]:
import numpy as np
import ctf

Generate random symmetric matrices and compute a reference answer using `numpy`.

In [11]:
n=4
def get_rand_sym_mat(n):
    A = np.random.random((n,n))-.5
    A = np.triu(A)
    A = A + A.T
    return A
A = get_rand_sym_mat(n)
B = get_rand_sym_mat(n)

C_ref = A @ B + B @ A

Compute intermediate tensor $$\hat{Z}_{ijk} = (A_{ij}+A_{ik}+A_{kj})\cdot(B_{ij}+B_{ik}+B_{kj})$$

In [16]:
A = ctf.astensor(A)
B = ctf.astensor(B)
C = ctf.tsr([n,n])

Zhat = ctf.tsr([n,n,n])
Zhat.i("ijk")<<(A.i("ij")+A.i("ik")+A.i("jk"))*(B.i("ij")+B.i("ik")+B.i("jk"))

Write a function to verify symmetry of tensors and apply it to $\hat{Z}$. We verify symmetry by checking that any transposition of modes leads to the same tensor. For a third order tensor, this implies that it has ${n+2\choose 3}$ unique elements. We will avoid accessing the diagonals of $\hat{Z}$, meaning they do not have to be computed, and only ${n\choose 3}$ elements of $\hat{Z}$ will be computed.

In [1]:
def verify_sym(A):
    for i in range(A.ndim):
        for j in range(i):
            istr1 = np.asarray(range(A.ndim))
            istr2 = istr1.copy()
            istr2[i] = istr1[j]
            istr2[j] = istr1[i]
            str1 = ''.join(str(s) for s in istr1)
            str2 = ''.join(str(s) for s in istr2)
            B = ctf.tsr(A.shape)
            B.i(str1) << A.i(str1)
            B.i(str2) << -1.0*B.i(str1)
            assert(B.norm2()/B.size < .000001)
#verify symmetry
verify_sym(A)
verify_sym(Zhat)
#zero out diagonal elements we will not use
Zhat.i("jii").scale(0.0) 
Zhat.i("iji").scale(0.0) 
Zhat.i("iij").scale(0.0)
print(Zhat)

NameError: name 'A' is not defined